In [ ]:
from pygbs import gbs
from pygbs import plotlyplot as gbp
import plotly.graph_objects as go

p = 5
filename = "../../../tests/in/e1098.dat"
crv_2d = gbs.bscurve2d_approx_from_points(filename,p,gbs.KnotsCalcMode.CHORD_LENGTH,1)

gbp.plot_bs_curve_2d([ crv_2d], ctrl_pts_on=True)

## Cut leading edge

In [ ]:
u1, u2 = crv_2d.bounds()

u1i = 0.43*(u2-u1)
u2i = 0.57*(u2-u1)

s1 = gbs.BSCurve2d(crv_2d)
s1.trim(u1, u1i)
s2 = gbs.BSCurve2d(crv_2d)
s2.trim(u2i,u2)

gbp.plot_bs_curve_2d([ s1, s2])

## Build C3 connection

In [ ]:
import numpy as np

ratio = 3.
du = gbs.dist(s1(u1i), s2(u2i)) * ratio

no = 3
no = min(p-1, no)
constraints = [(0, s1(u1i, o), o ) for o in range(1,no+1)] + [(du, s2(u2i, o), o ) for o in range(1,no+1)]
# constraints += [(0.5*du, [0.,0.], 0)]

c2_0 = gbs.interpolate(
    ( 0, s1(u1i)),
    (du, s2(u2i)),
    constraints,
    p
)
gbp.plot_bs_curve_2d([s1, s2, c2_0]).show()

## Join and simplify

In [ ]:
j1 = gbs.join(s1, c2_0)
j1.removeKnot(u1i,1e-2,p-1)
gbp.plot_bs_curve_2d([ j1, s2])

In [ ]:
j2 = gbs.join(j1, s2)
j2.removeKnot(u1i+du,1e-1,p-1)
gbp.plot_bs_curve_2d([j2])

## Check curvature

In [ ]:
from math import sqrt
from pytest import approx

def d_dm(curve, u):
    d1 = np.array(curve(u, 1))
    d1d1 = gbs.dot(d1, d1)
    return np.array(curve(u,1)) / sqrt(d1d1)

def d2_dm2(curve, u):
    d1 = np.array(curve(u, 1))
    d2 = np.array(curve(u, 2))

    d1d1 = gbs.dot(d1,d1)

    d1d2 = gbs.dot(d1, d2)

    return (d2 - d1 * d1d2 ) / d1d1

t1 = d_dm(s1, u1i)
t2 = d_dm(s2, u2i)
c1 = d2_dm2(s1, u1i)
c2 = d2_dm2(s2, u2i)

assert d_dm(s1, u1i)   == approx(d_dm(c2_0, 0))
assert d2_dm2(s1, u1i) == approx(d2_dm2(c2_0, 0))
assert d_dm(s2, u2i)   == approx(d_dm(c2_0, du))
assert d2_dm2(s2, u2i) == approx(d2_dm2(c2_0, du))
